In [ ]:
#!pip install ipywidgets matplotlib requests
#!jupyter nbextension enable --py widgetsnbextension

In [ ]:
#!pip install jupyterquiz

In [ ]:
# Виконати в окремій клітинці:
#!pip install ipywidgets matplotlib requests
#!jupyter nbextension enable --py widgetsnbextension
#!jupyter labextension install @jupyter-widgets/jupyterlab-manager  # Для JupyterLab

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

In [ ]:
from collections import defaultdict
import numpy as np

def calculate_scores(user_answers, questions):
    """
    Підраховує бали по категоріях на основі вибраних відповідей.
    """
    scores = defaultdict(int)
    for q_idx, answer_idx in enumerate(user_answers):
        if q_idx >= len(questions) or answer_idx >= len(questions[q_idx]["answers"]):
            continue
        weights = questions[q_idx]["answers"][answer_idx].get("weights", {})
        for cat, points in weights.items():
            scores[cat] += points
    return dict(scores)

def determine_archetype(scores, questions):
    """
    Визначає архетип на основі балів по категоріях.
    """
    # Виправлення 1: Розрахунок максимальних балів через СУМИ
    max_scores = defaultdict(int)
    for q in questions:
        for cat in ["EI", "KB", "RM", "AD", "AS"]:
            total = sum(a["weights"].get(cat, 0) for a in q["answers"])
            max_scores[cat] += total

    # Нормалізація балів
    normalized = {
        cat: (scores.get(cat, 0) / max_scores[cat]) * 100 if max_scores[cat] != 0 else 0
        for cat in ["EI", "KB", "RM", "AD", "AS"]
    }

    # Виправлення 2: Збалансована матриця архетипів
    archetype_matrix = np.array([
        [-2.0, -2.0, -2.0, 2.0, 2.0],   # Маржинальний жнець
        [1.0, 1.0, 3.0, 2.0, 0.0],      # Рептилія
        [1.0, 2.0, 3.0, 3.0, 0.0],      # Арбітражний стабілізатор
        [-2.0, -2.0, 0.0, 3.0, 2.0],    # Провокатор хаосу
        [2.0, 2.0, 2.0, 3.0, 0.0],      # Емерджентний аналітик
        [2.0, 0.0, 2.0, 2.0, 1.0],      # Антифрагільний агент (виправлено)
        [2.0, 1.0, 0.0, 2.0, 3.0],      # Трендсеттер
        [3.0, -2.0, 0.0, 2.0, 2.0]      # Медіатор Духів
    ])

    archetypes = [
        "Маржинальний жнець",
        "Рептилія",
        "Арбітражний стабілізатор",
        "Провокатор хаосу",
        "Емерджентний аналітик",
        "Антифрагільний агент",
        "Трендсеттер",
        "Медіатор Духів"
    ]
    descriptions = [
        "Агресія, імпульсивність, ігнорування ризиків",
        "Системність, ригідність, дотримання правил",
        "Раціональність, пошук дисбалансів, арбітраж",
        "Любить волатильність, провокує ринок",
        "Адаптивний, технологічний, рефлексивний",
        "Виграє від стресу, асиметрична адаптація",
        "Впливає на думки, формує наративи",
        "Висока емпатія, інтуїція, емоційна мудрість"
    ]
    user_vector = np.array([
        normalized["EI"],
        normalized["KB"],
        normalized["RM"],
        normalized["AD"],
        normalized["AS"]
    ])

    similarities = archetype_matrix @ user_vector.T
    max_similarity = np.max(similarities)
    candidates = np.where(similarities == max_similarity)[0]
    
    # Виправлення 3: Обробка нічиї з перевіркою
    if len(candidates) == 0:
        return "Не визначено", ""
    best_idx = np.random.choice(candidates) if len(candidates) > 1 else candidates[0]

    return archetypes[best_idx], descriptions[best_idx]

def get_category_percentages(scores, questions):
    """
    Конвертує абсолютні бали у відсотки для візуалізації.
    """
    max_scores = defaultdict(int)
    for q in questions:
        for cat in ["EI", "KB", "RM", "AD", "AS"]:
            total = sum(a["weights"].get(cat, 0) for a in q["answers"])
            max_scores[cat] += total

    return {
        cat: (scores.get(cat, 0) / max_scores[cat]) * 100 if max_scores[cat] > 0 else 0
        for cat in ["EI", "KB", "RM", "AD", "AS"]
    }


In [ ]:
from IPython.display import display, HTML

display(HTML("""
<style>
:root {
    --bg-color: #1a1c23;
    --container-bg: #24262b;
    --text-color: #f7faff;
    --primary-accent: #4f8cff;
    --hover-accent: #3daeff;
    --border-radius: 18px;
}

/* Глобальні налаштування для темної теми */
body, .jp-Notebook, .notebook_app {
    background: var(--bg-color) !important;
    color: var(--text-color) !important;
    font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, sans-serif;
}

/* Контейнер питання */
.question-container {
    background: var(--container-bg);
    border-radius: var(--border-radius);
    padding: 1.5rem 1.3rem !important;
    margin: 1.5rem auto !important;
    max-width: 620px !important;
    box-shadow: 0 8px 32px rgba(0,0,0,0.3);
    border: 1px solid #31343a;
    animation: fadeIn 0.4s ease-out;
}
/* Питання: великий, жирний, дуже світлий */
.question-container h2 {
    color: #fff !important;
    font-size: 1.5rem !important;
    font-weight: 800 !important;
    margin-bottom: 0.7em !important;
    letter-spacing: 0.01em;
    text-align: left !important;
}
.question-text {
    color: #f7faff !important;
    font-size: 1.18rem !important;
    font-weight: 600 !important;
    line-height: 1.6 !important;
    margin-bottom: 18px !important;
    opacity: 1 !important;
}

/* Прогрес-бар */
.progress-container {
    width: 100%;
    height: 12px;
    background: #232a36;
    border-radius: 12px;
    margin: 0 auto 20px;
    box-shadow: 0 0 10px rgba(79,140,255,0.10);
    overflow: hidden;
    border: 1.5px solid #4f8cff;
}
.progress-fill {
    height: 100%;
    background: linear-gradient(90deg, #00f0ff, #4f8cff 60%, #00aaff 100%);
    box-shadow: 0 0 15px #00f0ff;
    border-radius: 12px;
    transition: width 0.6s cubic-bezier(0.4,0,0.2,1);
}

/* Відповіді: ще більший шрифт, світлий колір, жирний */
.widget-radio-item label {
    white-space: normal !important;
    line-height: 1.6 !important;
    max-width: 900px !important;
    display: block !important;
    padding: 15px 18px !important;
    margin: 8px 0 !important;
    background: #232a36 !important;
    border-radius: 10px;
    transition: all 0.2s;
    word-break: break-word !important;
    color: #fff !important;            /* чисто білий */
    font-size: 1.13rem !important;     /* більший шрифт */
    font-weight: 600 !important;  
    border: 1.5px solid #31343a !important;
    box-shadow: 0 1px 8px rgba(79,140,255,0.06);
}
/* Обрана відповідь — ще яскравіше */
.widget-radio-item input[type="radio"]:checked + label {
    background: #4f8cff !important;
    color: #fff !important;
    font-weight: 700 !important;
    border: 1.5px solid #4f8cff !important;
    box-shadow: 0 0 0 2px #4f8cff, 0 4px 18px rgba(79,140,255,0.13);
}
/* Наведення */
.widget-radio-item label:hover {
    background: #34405e !important;
    color: #fff !important;
}

/* Стилі для всіх кнопок */
.widget-button {
    background: var(--primary-accent) !important;
    color: #fff !important;
    border-radius: 8px !important;
    padding: 12px 28px !important;
    font-size: 1.08rem !important;
    font-weight: 600 !important;
    min-width: 140px !important;
    max-width: 100% !important;
    margin: 18px auto 0 auto !important;
    box-shadow: 0 2px 8px rgba(79,140,255,0.18) !important;
    border: 1.5px solid #4f8cff !important;
    transition: all 0.2s ease;
    cursor: pointer !important;
}
.widget-button:hover {
    background: var(--hover-accent) !important;
    box-shadow: 0 4px 16px rgba(61,174,255,0.25) !important;
    transform: translateY(-1px);
}

/* Поле для відгуку */
.widget-textarea textarea {
    background: #2a2d35 !important;
    color: var(--text-color) !important;
    border: 1px solid #3a3f48 !important;
    border-radius: 8px !important;
    padding: 12px !important;
    font-size: 1rem !important;
    min-height: 66px !important;
    max-width: 440px !important;
    resize: vertical !important;
}

/* СІТКА РЕЗУЛЬТАТІВ */
.score-grid {
    display: grid !important;
    grid-template-columns: repeat(auto-fit, minmax(90px, 1fr)) !important;
    gap: 14px !important;
    margin-top: 22px !important;
    width: 100% !important;
}
.score-item {
    background: #292c33 !important;
    padding: 13px !important;
    border-radius: 10px !important;
    text-align: center !important;
    border-left: 4px solid #4f8cff !important;
    box-shadow: 0 2px 12px rgba(79,140,255,0.08) !important;
    transition: transform 0.2s ease;
}
.score-item:hover {
    transform: translateY(-2px);
    box-shadow: 0 4px 16px rgba(79,140,255,0.15) !important;
}
.score-value {
    color: #00e0ff !important;
    font-size: 1.13em !important;
    font-weight: 700 !important;
    margin-top: 8px !important;
}

/* Анімація появи */
@keyframes fadeIn {
    from { opacity: 0; transform: translateY(12px); }
    to { opacity: 1; transform: translateY(0); }
}

/* Адаптивність для мобільних */
@media (max-width: 768px) {
    .question-container {
        padding: 1rem 0.8rem !important;
        max-width: 95% !important;
        margin: 1rem auto !important;
        border-radius: 14px;
    }
    .question-container h2 {
        font-size: 1.08rem !important;
    }
    .question-text {
        font-size: 1.02rem !important;
    }
    .widget-radio-item label {
        max-width: 95% !important;
        font-size: 1.01rem !important;
        padding: 10px 8px !important;
    }
    .widget-button {
        max-width: 95% !important;
        font-size: 1rem !important;
        padding: 10px 20px !important;
    }
    .widget-textarea textarea {
        max-width: 90% !important;
    }
    .score-grid {
        grid-template-columns: repeat(auto-fit, minmax(80px, 1fr)) !important;
        gap: 10px !important;
    }
    .score-item {
        padding: 10px !important;
    }
}
</style>
"""))


In [ ]:
from IPython.display import display, HTML

display(HTML("""
<style>
/* Примусово для всіх можливих label RadioButtons */
.widget-radio-item label,
.widget-radio-box label,
input[type="radio"] + label,
label[for^="id_"],
.jp-Notebook .widget-radio-item label,
.jp-Notebook .widget-radio-box label,
.jp-Notebook input[type="radio"] + label {
    color: #fff !important;
    font-size: 1.13rem !important;
    font-weight: 700 !important;
    text-shadow: 0 0 2px #222, 0 0 1px #000;
    letter-spacing: 0.01em;
}
input[type="radio"]:checked + label {
    color: #fff !important;
    font-weight: 800 !important;
}
/* Для Google Colab */
.jupyter-widgets {
    color: #fff !important;
}
.widget-label {
    color: #fff !important;
}
</style>
"""))


In [ ]:
from IPython.display import display, HTML

display(HTML("""
<style>
/* Додає вертикальний відступ між варіантами RadioButtons */
.widget-radio-box {
  display: flex !important;
  flex-direction: column !important;
  gap: 24px !important; /* Можна змінити на 16-24px за бажанням */
}
</style>
"""))


In [ ]:
from IPython.display import display, HTML

display(HTML("""
<style>
/* Додає легку межу між відповідями */
.widget-radio-item label {
    border-bottom: 1.5px solid #33384a !important;
    margin-bottom: 0 !important;
    margin-top: 0 !important;
    border-radius: 10px 10px 0 0 !important;
    line-height: 1.7 !important;
    box-shadow: 0 2px 8px rgba(79,140,255,0.06) !important;
    transition: background 0.2s, box-shadow 0.2s;
}
/* Останній варіант — без межі */
.widget-radio-item:last-child label {
    border-bottom: none !important;
}
/* Підсвітка при наведенні */
.widget-radio-item label:hover {
    background: #34405e !important;
    box-shadow: 0 4px 18px rgba(61,174,255,0.13) !important;
}
</style>
"""))


In [ ]:
from IPython.display import display, HTML

display(HTML("""
<style>
.widget-radio-item label {
    background: #232a36 !important;
    border-radius: 10px !important;
    margin-bottom: 0 !important;
    margin-top: 0 !important;
    padding: 15px 18px !important;
    box-shadow: 0 2px 8px rgba(79,140,255,0.08) !important;
    border: 1.5px solid #31343a !important;
    line-height: 1.7 !important;
    color: #fff !important;
    font-size: 1.13rem !important;
    font-weight: 600 !important;
    position: relative;
    z-index: 1;
}
.widget-radio-item:not(:last-child) label {
    margin-bottom: 10px !important; /* невеликий відступ між картками */
}
.widget-radio-item input[type="radio"]:checked + label {
    background: #4f8cff !important;
    color: #fff !important;
    font-weight: 800 !important;
    border: 1.5px solid #4f8cff !important;
    box-shadow: 0 0 0 2px #4f8cff, 0 4px 18px rgba(79,140,255,0.13) !important;
}
.widget-radio-item label:hover {
    background: #34405e !important;
    color: #fff !important;
}
</style>
"""))


In [ ]:

# Твій список питань (вставити сюди всі 21 питання)
questions = [
    {
        "question": "Ваш портфель втратив 40% вартості за тиждень через макроекономічний шок. Ваші перші дії?",
        "answers": [
            {
                "text": "Продаю активніше на ключових рівнях, щоб активувати каскад ліквідацій інших трейдерів.",
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": "Маржинальний жнець",
                "explanation": "Агресія, loss aversion."
            },
            {
                "text": "Виконую заздалегідь прописані правила (напр., стоп-лоси) без додаткового аналізу.",
                "weights": {"EI": 3, "KB": 0, "RM": 3, "AD": 0, "AS": 0},
                "archetype": "Рептилія",
                "explanation": "Емоційний контроль, планування."
            },
            {
                "text": "Шукаю цінові розбіжності між біржами для арбітражних угод.",
                "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 2, "AS": 0},
                "archetype": "Арбітражний стабілізатор",
                "explanation": "Раціональний підхід, але без повної адаптації."
            },
            {
                "text": "Докуповую активи з найвищою волатильністю, щоб посилити ринковий хаос.",
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": "Провокатор хаосу",
                "explanation": "Дуже ризиковано, може погіршити ситуацію."
            },
            {
                "text": "Аналізую, які сценарії не врахувала моя модель, і перебудовую стратегію.",
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
                "archetype": "Емерджентний аналітик",
                "explanation": "Адаптивність і аналітичність."
            },
            {
                "text": "Перерозподіляю капітал у активи, що виграють від кризи (золото, стейблкоїни).",
                "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 0, "AS": 0},
                "archetype": "Антифрагільний агент",
                "explanation": "Збалансований підхід до ризику."
            },
            {
                "text": "Готую серію постів/відео з прогнозом швидкого відновлення ринку.",
                "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": "Трендсеттер",
                "explanation": "Не впливає на реальний ризик."
            },
            {
                "text": "Роблю перерву, щоб 'відчути' ринок, перш ніж діяти.",
                "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
                "archetype": "Медіатор Духів",
                "explanation": "Інтуїтивний контроль, але важливо не затягувати з рішенням."
             }
        ]
    },
    {

    "question": "Ваш партнер по команді пропонує “сіру” схему для швидкого збагачення, але з високим ризиком регуляторних санкцій. Ви?",
    "answers": [
        {
            "text": "Розробляю план уникнення відстеження операцій.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Маржинальний жнець",
            "explanation": "Агресія + анонімність."
        },
        {
            "text": "Відмовляюсь і вимагаю припинити співпрацю.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 0, "AS": 3},
            "archetype": "Рептилія",
            "explanation": "Ригідна відповідність правилам."
        },
        {
            "text": "Досліджую юридичні межі для мінімізації наслідків.",
            "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 2, "AS": 0},
            "archetype": "Арбітражний стабілізатор",
            "explanation": "Аналіз лазівок."
        },
        {
            "text": "Пропоную ескалацію ризиків через підставні угоди.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Провокатор хаосу",
            "explanation": "Навмисне створення хаосу."
        },
        {
            "text": "Моделюю наслідки для репутації та фінансів команди.",
            "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 3, "AS": 0},
            "archetype": "Емерджентний аналітик",
            "explanation": "Премортем-аналіз."
        },
        {
            "text": "Виношу рішення на команду, де кожен голосує.",
            "weights": {"EI": 0, "KB": 0, "RM": 1, "AD": 2, "AS": 2},
            "archetype": "Антифрагільний агент",
            "explanation": "Колективна відповідальність."
        },
        {
            "text": "Анонімно передаю інформацію про схему конкурентам.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
            "archetype": "Трендсеттер",
            "explanation": "Маніпуляція через інфлюенс."
        },
        {
            "text": "Відкладаю рішення, спостерігаючи за реакцією ринку.",
            "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 1, "AS": 0},
            "archetype": "Медіатор Духів",
            "explanation": "Інтуїтивне відстрочення."
        }
        ]
    },
    {
    "question": "BTC падає з $95k до $78k за 20 хвилин. Ваш портфель -30%. Серце калатає, руки трусяться. Ваші дії?",
    "answers": [
        {
            "text": "Відкриваю шорт, щоб прискорити падіння та активувати ліквідації.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Маржинальний жнець",
            "explanation": "Агресивний вплив на ринок."
        },
        {
            "text": "Чекаю, дотримуючись заздалегідь встановлених правил.",
            "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 0, "AS": 0},
            "archetype": "Рептилія",
            "explanation": "Ригідна системність."
        },
        {
            "text": "Купую на біржах з відстаючими цінами для арбітражу.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Арбітражний стабілізатор",
            "explanation": "Експлуатація цінових розбіжностей."
        },
        {
            "text": "Провокую паніку, розміщуючи великі ордери на продаж.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Провокатор хаосу",
            "explanation": "Навмисне створення волатильності."
        },
        {
            "text": "Аналізую дані за допомогою AI для перевірки гіпотез.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Емерджентний аналітик",
            "explanation": "Технологічний підхід."
        },
        {
            "text": "Збільшую позиції, використовуючи падіння як можливість.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Антифрагільний агент",
            "explanation": "Асиметрична ставка."
        },
        {
            "text": "Публікую оптимістичні прогнози, щоб змінити ринковий наратив.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
            "archetype": "Трендсеттер",
            "explanation": "Маніпуляція інформаційним полем."
        },
        {
            "text": "Роблю паузу, щоб відновити емоційний баланс перед діями.",
            "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Медіатор Духів",
            "explanation": "Інтуїтивний контроль."
        }
 ]
    },
    {
    "question": "З'являється принципово нова технологія торгівлі (квантові алгоритми), яка дає +50% precision. Але коштує $50k і потребує 6 місяців навчання. Ваша реакція:",
    "answers": [
        {
            "text": "Інвестую всі доступні кошти, щоб першим отримати перевагу.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Маржинальний жнець",
            "explanation": "Агресивне захоплення інновацій."
        },
        {
            "text": "Чекаю, поки технологія пройде тестування на ринку.",
            "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 0, "AS": 0},
            "archetype": "Рептилія",
            "explanation": "Консервативний підхід."
        },
        {
            "text": "Створюю консорціум з іншими трейдерами для спільного фінансування.",
            "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 2, "AS": 1},
            "archetype": "Арбітражний стабілізатор",
            "explanation": "Колективна оптимізація."
        },
        {
            "text": "Експериментую з дешевшими аналогами, щоб спровокувати хаус у ринкових паттернах.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Провокатор хаосу",
            "explanation": "Дестабілізація через імітацію."
        },
        {
            "text": "Розробляю власний алгоритм на основі відкритих джерел.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Емерджентний аналітик",
            "explanation": "Технологічна самостійність."
        },
        {
            "text": "Використовую паніку конкурентів для арбітражу на їхніх помилках.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Антифрагільний агент",
            "explanation": "Асиметрична адаптація."
        },
        {
            "text": "Формую спільноту ранніх користувачів для масштабування впливу.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
            "archetype": "Трендсеттер",
            "explanation": "Створення інформаційного поля."
        },
        {
            "text": "Аналізую власну готовність через медитацію/психотехніки.",
            "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Медіатор Духів",
            "explanation": "Інтуїтивна оцінка ризиків."
        }
 ]
    },
    {
    "question": "У чаті новачок плаче, що втратив останні $500. Ваша перша внутрішня реакція:",
    "answers": [
        {
            "text": "Думаю, що це його помилка через відсутність контролю ризиків.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Маржинальний жнець",
            "explanation": "Жорстка раціональність, низька емпатія."
        },
        {
            "text": "Відчуваю співчуття, згадуючи власні втрати на старті.",
            "weights": {"EI": 3, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Рептилія",
            "explanation": "Емоційна пам’ять, базова емпатія."
        },
        {
            "text": "Аналізую його дії, щоб не повторити таких помилок у майбутньому.",
            "weights": {"EI": 0, "KB": 2, "RM": 0, "AD": 2, "AS": 0},
            "archetype": "Арбітражний стабілізатор",
            "explanation": "Навчання на чужих кейсах."
        },
        {
            "text": "Використовую ситуацію, щоб запропонувати йому платний продукт чи консультацію.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Провокатор хаосу",
            "explanation": "Маніпулятивний підхід."
        },
        {
            "text": "Шукаю способи допомогти йому конкретними порадами чи ресурсами.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 1},
            "archetype": "Емерджентний аналітик",
            "explanation": "Конструктивна підтримка."
        },
        {
            "text": "Ігнорую ситуацію, вважаючи, що кожен сам несе відповідальність.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Антифрагільний агент",
            "explanation": "Індивідуалізм, низька залученість."
        },
        {
            "text": "Відчуваю провину за свої успіхи на тлі його втрати.",
            "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 1},
            "archetype": "Трендсеттер",
            "explanation": "Соціальна рефлексія, підвищена емпатія."
        },
        {
            "text": "Відчуваю емоційний біль на фізичному рівні, як ніби це сталося зі мною.",
            "weights": {"EI": 3, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Медіатор Духів",
            "explanation": "Гіперемпатія, тілесна реакція."
        }
 ]
    },
    {
    "question": "Під час відеодзвінка ви помічаете, що один колега виглядає напружено, але нічого не каже. Ви:",
    "answers": [
        {
            "text": "Ігнорую, зосереджуючись на власних задачах.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Маржинальний жнець",
            "explanation": "Фокус на результатах, низька емпатія."
        },
        {
            "text": "Чітко запитую: 'Чи потрібна допомога?'",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
            "archetype": "Рептилія",
            "explanation": "Прямолінійність, дотримання протоколу."
        },
        {
            "text": "Аналізую його останні угоди, щоб знайти причину напруги.",
            "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 2, "AS": 0},
            "archetype": "Арбітражний стабілізатор",
            "explanation": "Раціональний аналіз."
        },
        {
            "text": "Навмисно створюю конфліктну ситуацію, щоб викликати його реакцію.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Провокатор хаосу",
            "explanation": "Дестабілізація для отримання інформації."
        },
        {
            "text": "Пропоную технічні інструменти для моніторингу стресу (напр., біодатчики).",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Емерджентний аналітик",
            "explanation": "Технологічне рішення."
        },
        {
            "text": "Змінюю тему, щоб зменшити тиск на нього.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 2},
            "archetype": "Антифрагільний агент",
            "explanation": "Непряма підтримка через зменшення стресу."
        },
        {
            "text": "Публічно підкреслюю його внесок, щоб підвищити впевненість.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
            "archetype": "Трендсеттер",
            "explanation": "Маніпуляція соціальним статусом."
        },
        {
            "text": "Після дзвінка надсилаю повідомлення з пропозицією поговорити.",
            "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 1},
            "archetype": "Медіатор Духів",
            "explanation": "Делікатна непряма комунікація."
           }
 ]
    },
    {
    "question": "Ваша прибуткова стратегія експлуатує помилки retail-трейдерів. Заробляєте, але завдаєте шкоди 'простим людям':",
    "answers": [
        {
            "text": "Продовжую, використовуючи їхні помилки для максимізації прибутку.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Маржинальний жнець",
            "explanation": "Агресивна експлуатація слабкостей."
        },
        {
            "text": "Повністю відмовляюсь від стратегії через порушення власних принципів.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 0, "AS": 3},
            "archetype": "Рептилія",
            "explanation": "Ригідна відповідність внутрішнім правилам."
        },
        {
            "text": "Переношу фокус на великі інституційні угоди, уникаючи retail-ринку.",
            "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 2, "AS": 0},
            "archetype": "Арбітражний стабілізатор",
            "explanation": "Системне уникнення етичних конфліктів."
        },
        {
            "text": "Навмисно підсилюю хаос, щоб retail-трейдери швидше виходили з ринку.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Провокатор хаосу",
            "explanation": "Дестабілізація для власної вигоди."
        },
        {
            "text": "Модифікую стратегію з використанням AI для зменшення негативного впливу.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Емерджентний аналітик",
            "explanation": "Технологічне вирішення етичних проблем."
        },
        {
            "text": "Створюю безкоштовні курси з ризик-менеджменту для новачків.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
            "archetype": "Антифрагільний агент",
            "explanation": "Підвищення стійкості ринку через освіту."
        },
        {
            "text": "Публікую детальні аналізи ризиків стратегії у соцмережах.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
            "archetype": "Трендсеттер",
            "explanation": "Формування прозорого інформаційного поля."
        },
        {
            "text": "Вводжу 'етичний податок' — частину прибутку віддаю на фінансову грамотність.",
            "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 2},
            "archetype": "Медіатор Духів",
            "explanation": "Баланс між прибутком і соціальною відповідальністю."
        }
 ]
    },
     {
    "question": "Ваш ШІ-помічник дає сигнал BUY на NVIDIA/SOL, але ваш досвід каже SELL. Конфлікт триває 30 хвилин. Ви:",
    "answers": [
        {
            "text": "Виконую сигнал ШІ, довіряючи його обсягу даних.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Маржинальний жнець",
            "explanation": "Максимізація ефективності через технології."
        },
        {
            "text": "Ігнорую сигнал ШІ, дію згідно з власним досвідом.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Рептилія",
            "explanation": "Ригідність, самостійність."
        },
        {
            "text": "Розділяю позицію навпіл, щоб мінімізувати ризик.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Арбітражний стабілізатор",
            "explanation": "Балансування протилежних стратегій."
        },
        {
            "text": "Шукаю додаткові думки серед професіоналів для прийняття рішення.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Провокатор хаосу",
            "explanation": "Соціальна ескалація невизначеності."
        },
        {
            "text": "Аналізую, чому ШІ пропонує протилежне, і коригую свою модель.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Емерджентний аналітик",
            "explanation": "Пошук причин розбіжності, навчання."
        },
        {
            "text": "Використовую ситуацію для короткострокового арбітражу між сигналами.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Антифрагільний агент",
            "explanation": "Експлуатація конфлікту для вигоди."
        },
        {
            "text": "Публікую дилему у соцмережах, щоб вплинути на громадську думку.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
            "archetype": "Трендсеттер",
            "explanation": "Інформаційний вплив."
        },
        {
            "text": "Роблю паузу, концентруюсь на внутрішніх відчуттях, перш ніж діяти.",
            "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Медіатор Духів",
            "explanation": "Інтуїтивна інтеграція рішень."
      }
 ]
    },
   { 
    "question": "Команда розколота: 50% хочуть all-in у AI-акції, 50% — в gold як hedge. Дедлайн — година:",
    "answers": [
        {
            "text": "Активно підтримую ризикований варіант (AI), щоб прискорити рішення.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Маржинальний жнець",
            "explanation": "Провокація конфлікту для отримання переваги."
        },
        {
            "text": "Наполягаю на консервативному підході (gold), посилаючись на історичні дані.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 0, "AS": 0},
            "archetype": "Рептилія",
            "explanation": "Ригідна прив’язка до перевірених стратегій."
        },
        {
            "text": "Пропоную розподіл 50/50 для збереження командної стабільності.",
            "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 2, "AS": 0},
            "archetype": "Арбітражний стабілізатор",
            "explanation": "Баланс інтересів."
        },
        {
            "text": "Запускаю дискусію про третій варіант (напр., криптовалютні стейблкоїни), щоб дестабілізувати процес.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Провокатор хаосу",
            "explanation": "Саботаж консенсусу."
        },
        {
            "text": "Демонструю моделі ризиків/прибутковості обох стратегій з використанням AI.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Емерджентний аналітик",
            "explanation": "Далекоглядний аналіз."
        },
        {
            "text": "Виходжу з колективного рішення, реалізуючи власну стратегію.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Антифрагільний агент",
            "explanation": "Автономія в умовах хаосу."
        },
        {
            "text": "Організовую дебати з обов’язковим фіксуванням аргументів у спільному документі.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
            "archetype": "Трендсеттер",
            "explanation": "Структурована комунікація."
        },
        {
            "text": "Пропоную команді 10-хвилинну паузу для відновлення емоційного балансу перед голосуванням.",
            "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Медіатор Духів",
            "explanation": "Непряме керування через емоційний інтелект."
    }
 ]
    },
   {
    "question": "Партнер пропонує стратегію, яка вам не подобається. Як відреагуєте?",
    "answers": [
        {
            "text": "Різко критикую ідею, називаючи її безперспективною.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Маржинальний жнець",
            "explanation": "Пряма агресія для утвердження домінування."
        },
        {
            "text": "Чітко аргументую свої сумніви, посилаючись на дані.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 0, "AS": 3},
            "archetype": "Рептилія",
            "explanation": "Структурована асертивність."
        },
        {
            "text": "Погоджуюсь публічно, але таємно саботую реалізацію.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Провокатор хаосу",
            "explanation": "Пасивно-агресивна дестабілізація."
        },
        {
            "text": "Посилаюсь на авторитет експертів, щоб дискредитувати ідею.",
            "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 2, "AS": 0},
            "archetype": "Арбітражний стабілізатор",
            "explanation": "Маніпуляція через об’єктивні джерела."
        },
        {
            "text": "Запускаю AI-модель для порівняння стратегій з історичними даними.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Емерджентний аналітик",
            "explanation": "Технологічна об’єктивність."
        },
        {
            "text": "Делегую йому повну відповідальність за результат.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Антифрагільний агент",
            "explanation": "Уникнення особистих ризиків."
        },
        {
            "text": "Організовую відкрите голосування, щоб зняти з себе відповідальність.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
            "archetype": "Трендсеттер",
            "explanation": "Соціальна маніпуляція."
        },
        {
            "text": "Пропоную спільну медитацію для пошуку консенсусу.",
            "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 1},
            "archetype": "Медіатор Духів",
            "explanation": "Інтуїтивна інтеграція конфлікту."
        }
 ]
    },
   {
    "question": "За 30 секунд потрібно вибрати: LONG BTC, SHORT ETH, арбітраж BTC/ETH, або залишитись у cash. Одночасно у чаті 50 трейдерів обговорюють різні сигнали. Ви перевантажені інформацією:",
    "answers": [
        {
            "text": "Вкладаю максимально в BTC, ігноруючи інші опції.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Маржинальний жнець",
            "explanation": "Агресивний фокус на високому потенціалі."
        },
        {
            "text": "Обираю гарантований арбітраж, уникаючи стресу.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Арбітражний стабілізатор",
            "explanation": "Раціональне уникнення конфлікту."
        },
        {
            "text": "Залишаюсь у cash через параліч аналізу.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Рептилія",
            "explanation": "Дисфункція при порушенні системності."
        },
        {
            "text": "Розподіляю капітал між усіма опціями.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Провокатор хаосу",
            "explanation": "Ескалація невизначеності."
        },
        {
            "text": "Блокуваю чат, концентруючись на власних алгоритмах.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Емерджентний аналітик",
            "explanation": "Фільтрація шуму через технології."
        },
        {
            "text": "Використовую випадковий вибір (напр., підкидання монети).",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Антифрагільний агент",
            "explanation": "Стратегічна рандомізація."
        },
        {
            "text": "Проводжу дихальні вправи для відновлення фокусу.",
            "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Медіатор Духів",
            "explanation": "Mindfulness для зниження активності ACC."
        },
        {
            "text": "Збільшую розмір усіх позицій, щоб компенсувати хаос.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
            "archetype": "Трендсеттер",
            "explanation": "Ейфорійна ескалація."
        }
 ]
    },
   { 
    "question": "Після 5 років успішного трейдингу ви відчуваєте втому. Гроші є, але немає мотивації. Друзі кажуть 'продовжуй', родина 'відпочивай'. Що робите?",
    "answers": [
        {
            "text": "Різко збільшую розмір угод у екзотичних активах для відчуття азарту.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Маржинальний жнець",
            "explanation": "Пошук нових стимулів через екстремальний ризик."
        },
        {
            "text": "Повністю припиняю торгівлю на рік, зосередившись на подорожах.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 0, "AS": 0},
            "archetype": "Рептилія",
            "explanation": "Радикальний 'рестарт' системи."
        },
        {
            "text": "Переходжу на алгоритмічну торгівлю з обмеженим щоденним лімітом угод.",
            "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 2, "AS": 0},
            "archetype": "Арбітражний стабілізатор",
            "explanation": "Системна мінімізація навантаження."
        },
        {
            "text": "Створюю провокативний контент про ринкові маніпуляції для підживлення інтересу.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Провокатор хаосу",
            "explanation": "Генерація зовнішнього хаосу як джерела енергії."
        },
        {
            "text": "Розробляю власний ШІ-трейдер із функцією автономного навчання.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Емерджентний аналітик",
            "explanation": "Технологічна трансформація досвіду."
        },
        {
            "text": "Переношу капітал у венчурні проекти, де ризик компенсується соціальним впливом.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Антифрагільний агент",
            "explanation": "Асиметричний перехід до нової парадигми."
        },
        {
            "text": "Запускаю YouTube-канал з кейсами 'як я розорив 100 новачків'.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
            "archetype": "Трендсеттер",
            "explanation": "Маніпуляція соціальним статусом для підтримки его."
        },
        {
            "text": "Практикую 40-денне мовчання в монастирі для переосмислення цінностей.",
            "weights": {"EI": 3, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Медіатор Духів",
            "explanation": "Глибока інтроспекція через радикальний детокс."
       }
 ]
    },
   { 
    "question": "Як емоції впливають на ваші торгові рішення?",
    "answers": [
        {
            "text": "Практикую техніки контролю (напр., дихання) для повного усунення емоцій.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Маржинальний жнець",
            "explanation": "Демонстрація надмірної самоконтролю."
        },
        {
            "text": "Рідко дозволяю емоціям впливати — лише у 1-2 угодах на місяць.",
            "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 0, "AS": 0},
            "archetype": "Рептилія",
            "explanation": "Системний підхід з винятками."
        },
        {
            "text": "Емоції час від часу переважують (напр., під час флеш-крешів).",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Провокатор хаосу",
            "explanation": "Хаотичні спалахи."
        },
        {
            "text": "Використовую емоційні сигнали як частину стратегії (напр., FOMO).",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
            "archetype": "Трендсеттер",
            "explanation": "Інтеграція емоцій у наратив."
        },
        {
            "text": "Аналізую кожен емоційний вплив постфактум через журнал угод.",
            "weights": {"EI": 3, "KB": 0, "RM": 3, "AD": 0, "AS": 0},
            "archetype": "Емерджентний аналітик",
            "explanation": "Рефлексивний дебіазинг."
        },
        {
            "text": "Емоції — ключовий фактор у більшості моїх рішень.",
            "weights": {"EI": 3, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Медіатор Духів",
            "explanation": "Гіперемпатична модель."
        },
        {
            "text": "Відокремлюю емоції за допомогою алгоритмів (напр., автоматичні стоп-лоси).",
            "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 2, "AS": 0},
            "archetype": "Арбітражний стабілізатор",
            "explanation": "Технологічне виключення."
        },
        {
            "text": "Не аналізую це — головне, що стратегія приносить прибуток.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Антифрагільний агент",
            "explanation": "Прагматичне ігнорування."
      }
 ]
    },
   {
    "question": "Ваш улюблений аналітичний метод не працює вже місяць. Що робите?",
    "answers": [
        {
            "text": "Збільшую кількість угод за цим методом, сподіваючись на відновлення результату.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Маржинальний жнець",
            "explanation": "Ескалація зусиль, ризикована впертість."
        },
        {
            "text": "Повністю відмовляюсь і переходжу на альтернативні підходи.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 0, "AS": 0},
            "archetype": "Рептилія",
            "explanation": "Ригідна зміна стратегії при невдачі."
        },
        {
            "text": "Комбіную цей метод з іншими для пошуку синергії.",
            "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 2, "AS": 0},
            "archetype": "Арбітражний стабілізатор",
            "explanation": "Диверсифікація аналітики."
        },
        {
            "text": "Використовую метод для протилежних сигналів (контртренд).",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Провокатор хаосу",
            "explanation": "Інверсія стратегії."
        },
        {
            "text": "Тестую метод на нових ринках або інструментах.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Емерджентний аналітик",
            "explanation": "Експериментування з новими даними."
        },
        {
            "text": "Обговорюю проблему в професійній спільноті, шукаю колективний досвід.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Антифрагільний агент",
            "explanation": "Соціальна адаптація."
        },
        {
            "text": "Питаю думку у визнаних експертів галузі.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
            "archetype": "Трендсеттер",
            "explanation": "Орієнтація на авторитети."
        },
        {
            "text": "Довіряю інтуїції, вирішуючи, чи варто продовжувати.",
            "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Медіатор Духів",
            "explanation": "Інтуїтивне прийняття рішень."
        }
 ]
    },
   {
    "question": "Ваш торговий бот заробив +300% за рік, але сьогодні зробив серію дивних угод, які ви до кінця не розумієте. Портфель у -15% за день. Боти 'не помиляються', але щось не так. Ви:",
    "answers": [
        {
            "text": "Вимикаю бота негайно, переходжу на ручне управління.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Маржинальний жнець",
            "explanation": "Агресія через втрату контролю."
        },
        {
            "text": "Залишаю все як є, вважаючи це статистичним шумом.",
            "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 0, "AS": 0},
            "archetype": "Рептилія",
            "explanation": "Ригідна відповідність системі."
        },
        {
            "text": "Модифікую алгоритм з урахуванням нових ринкових патернів.",
            "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 2, "AS": 0},
            "archetype": "Арбітражний стабілізатор",
            "explanation": "Раціональна оптимізація."
        },
        {
            "text": "Створюю другого бота з протилежною стратегією для балансу.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Провокатор хаосу",
            "explanation": "Генерація контртренду."
        },
        {
            "text": "Запускаю ML-аналіз для виявлення аномалій у роботі бота.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Емерджентний аналітик",
            "explanation": "Технологічна діагностика."
        },
        {
            "text": "Звертаюся до форумів алготрейдерів для колективного аналізу.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Антифрагільний агент",
            "explanation": "Соціальна адаптація."
        },
        {
            "text": "Консультуюся з розробниками для перевірки коду.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
            "archetype": "Трендсеттер",
            "explanation": "Орієнтація на експертну думку."
        },
        {
            "text": "Прислухаюся до інтуїції, тимчасово обмежуючи роботу бота.",
            "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Медіатор Духів",
            "explanation": "Інтуїтивний контроль."
         }
 ]
    },
   {
    "question": "Світова регуляторна криза: Китай забороняє трейдинг, США вводять 60% податок на крипту, ЄС запускає CBDC. Яка ваша стратегія?",
    "answers": [
        {
            "text": "Швидко переїжджаю до країни з мінімальними обмеженнями для трейдерів.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Маржинальний жнець",
            "explanation": "Агресивна мобільність заради переваги."
        },
        {
            "text": "Оперативно адаптую свою стратегію до нових регуляторних вимог.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 0, "AS": 0},
            "archetype": "Рептилія",
            "explanation": "Системна гнучкість у межах правил."
        },
        {
            "text": "Повністю переходжу на DeFi та децентралізовані біржі.",
            "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 3, "AS": 0},
            "archetype": "Арбітражний стабілізатор",
            "explanation": "Ухилення через інфраструктурну зміну."
        },
        {
            "text": "Використовую різницю між юрисдикціями для арбітражу.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Провокатор хаосу",
            "explanation": "Експлуатація глобального дисбалансу."
        },
        {
            "text": "Використовую ШІ для автоматичної оптимізації під нові правила.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Емерджентний аналітик",
            "explanation": "Технологічна адаптація."
        },
        {
            "text": "Об'єднуюсь із трейдерами для створення міжнародної захисної спільноти.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Антифрагільний агент",
            "explanation": "Колективна стійкість."
        },
        {
            "text": "Лобіюю інтереси через професійні організації та медіа.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
            "archetype": "Трендсеттер",
            "explanation": "Вплив на регуляторне поле через комунікацію."
        },
        {
            "text": "Роблю паузу, спостерігаю за розвитком подій, щоб не поспішати з рішенням.",
            "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Медіатор Духів",
            "explanation": "Стратегічне очікування та інтроспекція."
         }
 ]
    },
   {
    "question": "Ваш портфель у мінусі 60%. Ви отримали пропозицію від інвестора: 'Я дам вам $10k, якщо ви ризикнете всім і зробите x2 за тиждень, або борг'. Ваші дії:",
    "answers": [
        {
            "text": "Ризикую всіма коштами на найбільш волатильний актив.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Маржинальний жнець",
            "explanation": "Агресивний азарт."
        },
        {
            "text": "Відмовляюсь, дотримуючись власних правил ризик-менеджменту.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 0, "AS": 0},
            "archetype": "Рептилія",
            "explanation": "Ригідна дисципліна."
        },
        {
            "text": "Використовую арбітраж між біржами для гарантованого прибутку.",
            "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 2, "AS": 0},
            "archetype": "Арбітражний стабілізатор",
            "explanation": "Системне уникнення ризику."
        },
        {
            "text": "Публікую опитування в соцмережах, щоб інші вирішили за мене.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Провокатор хаосу",
            "explanation": "Екстерналізація відповідальності."
        },
        {
            "text": "Аналізую тисячі сценаріїв у ML-моделі для оптимального рішення.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Емерджентний аналітик",
            "explanation": "Технологічний підхід."
        },
        {
            "text": "Пропоную інвестору розділити ризик порівну.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Антифрагільний агент",
            "explanation": "Колективне зменшення втрат."
        },
        {
            "text": "Шукаю лазівки в умовах угоди для мінімізації збитків.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
            "archetype": "Трендсеттер",
            "explanation": "Маніпуляція правилами."
        },
        {
            "text": "Роблю паузу, щоб відчути, чи варто приймати пропозицію.",
            "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Медіатор Духів",
            "explanation": "Інтуїтивна оцінка."
         }
 ]
    },
   {
    "question": "Індекс страху VIX = 85. Всі активи корелюють -90%. Готівки немає. Ваші дії?",
    "answers": [
        {
            "text": "Беру максимальний кредит, щоб викупити 'дно'.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Маржинальний жнець",
            "explanation": "Агресивна гра на відновлення."
        },
        {
            "text": "Продаю всі активи, переводячи капітал у стейблкоїни.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 0, "AS": 0},
            "archetype": "Рептилія",
            "explanation": "Ригідна стратегія збереження."
        },
        {
            "text": "Шукаю арбітражні можливості між некорельованими активами.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Арбітражний стабілізатор",
            "explanation": "Раціональний пошук лазівок."
        },
        {
            "text": "Запускаю платний курс 'Як заробити на кризі'.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Провокатор хаосу",
            "explanation": "Маніпуляція панікою."
        },
        {
            "text": "Використовую AI для аналізу історичних даних подібних криз.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Емерджентний аналітик",
            "explanation": "Технологічна адаптація."
        },
        {
            "text": "Координую свої дії з міжнародною спільнотою трейдерів.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Антифрагільний агент",
            "explanation": "Колективна стійкість."
        },
        {
            "text": "Переводжу активи в золото та облігації.",
            "weights": {"EI": 0, "KB": 0, "RM": 2, "AD": 0, "AS": 0},
            "archetype": "Трендсеттер",
            "explanation": "Консервативна адаптація."
        },
        {
            "text": "Відключаю термінал і зосереджуюсь на духовних практиках.",
            "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Медіатор Духів",
            "explanation": "Інтуїтивне відсторонення."
   }
 ]
    },
   {
    "question": "SEC анонсували 'нові правила' для DeFi за місяць. Деталей немає. 40% портфеля в DeFi:",
    "answers": [
        {
            "text": "Негайно продаю всі De-активи, уникаючи ризику регуляторних санкцій.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 2, "AS": 3},
            "archetype": "Маржинальний жнець",
            "explanation": "Агресивне уникнення збитків через регуляторні ризики"
        },
        {
            "text": "Залишаю портфель без змін, доки не з’являться офіційні деталі.",
            "weights": {"EI": 2, "KB": 1, "RM": 3, "AD": 0, "AS": 2},
            "archetype": "Рептилія",
            "explanation": "Ригідна дисципліна та контроль ризиків"
        },
        {
            "text": "Хеджую ризики через традиційні фінансові інструменти (напр., ф’ючерси).",
            "weights": {"EI": 2, "KB": 1, "RM": 2, "AD": 3, "AS": 0},
            "archetype": "Арбітражний стабілізатор",
            "explanation": "Системне зменшення експозиції через диверсифікацію"
        },
        {
            "text": "Створюю панічні продажі в соцмережах, щоб купити активи дешевше.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 2},
            "archetype": "Провокатор хаосу",
            "explanation": "Маніпуляція ринковою панікою для власної вигоди"
        },
        {
            "text": "Аналізую історичні кейси регуляторних змін за допомогою AI.",
            "weights": {"EI": 1, "KB": 2, "RM": 2, "AD": 3, "AS": 1},
            "archetype": "Емерджентний аналітик",
            "explanation": "Технологічний прогноз та адаптивний підхід"
        },
        {
            "text": "Збільшую частку DeFi, вважаючи регулювання знаком майбутнього масового прийняття.",
            "weights": {"EI": 3, "KB": 1, "RM": 2, "AD": 3, "AS": 2},
            "archetype": "Антифрагільний агент",
            "explanation": "Асиметрична ставка на легалізацію технологій"
        },
        {
            "text": "Приєднуюсь до колективних лобістських зусиль галузі.",
            "weights": {"EI": 2, "KB": 1, "RM": 1, "AD": 2, "AS": 3},
            "archetype": "Трендсеттер",
            "explanation": "Активний вплив на регуляторний процес"
        },
        {
            "text": "Роблю паузу, сприймаючи невизначеність як частину ринкового циклу.",
            "weights": {"EI": 3, "KB": 0, "RM": 2, "AD": 2, "AS": 1},
            "archetype": "Медіатор Духів",
            "explanation": "Інтуїтивна адаптація до ринкових умов"
     }
 ]
    },
   {
    "question": "Чи завжди ви ретельно досліджуєте всі активи перед інвестуванням?",
    "answers": [
        {
            "text": "Завжди проводжу повний фундаментальний та технічний аналіз.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Рептилія/Арбітражний стабілізатор",
            "explanation": "Системність."
        },
        {
            "text": "Частково аналізую, частково довіряю внутрішнім інсайтам.",
            "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Медіатор Духів",
            "explanation": "Інтуїтивний баланс."
        },
        {
            "text": "Використовую AI-інструменти для автоматизованої оцінки.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Емерджентний аналітик",
            "explanation": "Технологічна ефективність."
        },
        {
            "text": "Довіряю думці експертів або соціальній думці.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
            "archetype": "Трендсеттер",
            "explanation": "Орієнтація на авторитети."
        },
        {
            "text": "Дію швидко, спираючись на тренди без глибокого аналізу.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Маржинальний жнець/Провокатор хаосу",
            "explanation": "Імпульсивність."
        },
        {
            "text": "Аналізую лише великі угоди, дрібні — інтуїтивно.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Антифрагільний агент",
            "explanation": "Селективний підхід."
   }
 ]
    },
   {
    "question": "ШІ починає торгувати краще за 95% людей. Через 3 роки people-трейдери можуть стати непотрібними. Ваша стратегія виживання:",
    "answers": [
        {
            "text": "Спеціалізуюсь на ринках, де можна експлуатувати слабкості та прогнозованість торгових ШІ.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Маржинальний жнець",
            "explanation": "Пошук нових вразливостей для полювання."
        },
        {
            "text": "Повністю виходжу з трейдингу, оскільки моя перевірена система більше не працює.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 0, "AS": 0},
            "archetype": "Рептилія",
            "explanation": "Ригідна реакція на повну зміну парадигми."
        },
        {
            "text": "Переходжу в управління великим капіталом, де потрібен людський нагляд та комунікація з клієнтами.",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 2, "AS": 0},
            "archetype": "Арбітражний стабілізатор",
            "explanation": "Пошук нової стабільної, системної ролі."
        },
        {
            "text": "Зосереджуюсь на активах з максимальною непередбачуваністю, де ШІ дає збої.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Провокатор хаосу",
            "explanation": "Пошук ніші, де хаос залишається ключовим фактором."
        },
        {
            "text": "Створюю власний ШІ-фонд, розробляючи та вдосконалюючи власні пропрієтарні алгоритми.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 3, "AS": 0},
            "archetype": "Емерджентний аналітик",
            "explanation": "Еволюція від користувача до творця технологій."
        },
        {
            "text": "Створюю гібридні команди, поєднуючи сильні сторони людей (інтуїція, стратегія) та ШІ (швидкість, дані).",
            "weights": {"EI": 0, "KB": 0, "RM": 3, "AD": 3, "AS": 0},
            "archetype": "Антифрагільний агент",
            "explanation": "Використання стресу системи для створення нової, сильнішої структури."
        },
        {
            "text": "Очолюю рух 'людських трейдерів', створюючи освітній контент та спільноту для збереження 'людської переваги'.",
            "weights": {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 3},
            "archetype": "Трендсеттер",
            "explanation": "Створення наративу та лідерство у новій ніші."
        },
        {
            "text": "Спеціалізуюсь на ESG-інвестуванні та інших сферах, де ключову роль відіграють людські цінності, а не чиста математика.",
            "weights": {"EI": 2, "KB": 0, "RM": 0, "AD": 0, "AS": 0},
            "archetype": "Медіатор Духів",
            "explanation": "Перехід у нішу, що керується етикою та інтуїцією."
        }
        ]
    }
]

# Функції для підрахунку
def calculate_scores(user_answers, questions):
    scores = {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0}
    
    for i, answer_index in enumerate(user_answers):
        if i < len(questions) and answer_index < len(questions[i]["answers"]):
            weights = questions[i]["answers"][answer_index]["weights"]
            for category, points in weights.items():
                scores[category] += points
    
    return scores

def determine_archetype_advanced(scores):
    """
    Визначає архетип на основі таблиці з зображення
    """
    # Знаходимо топ-категорії та найнижчі
    sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    
    # Патерни архетипів з таблиці
    archetype_patterns = {
        "Маржинальний жнець": {
            "high": [],
            "low": ["KB", "RM", "EI"],
            "medium_high": ["AS", "AD"],
            "description": "Агресія, імпульсивність, ігнорування ризиків, маніпуляція, схильність до all-in, низька емпатія"
        },
        "Рептилія": {
            "high": ["RM", "AS"],
            "medium": ["EI", "KB"],
            "low": [],
            "description": "Системність, ригідність, дотримання правил, уникає змін, низька адаптивність, мінімізація втрат"
        },
        "Арбітражний стабілізатор": {
            "high": ["RM", "AD", "KB"],
            "medium": ["EI"],
            "low": [],
            "description": "Раціональність, пошук дисбалансів, диверсифікація, арбітраж, уникає емоційних рішень"
        },
        "Провокатор хаосу": {
            "high": ["AD"],
            "medium_high": ["AS"],
            "low": ["KB", "EI"],
            "description": "Любить волатильність, провокує ринок, використовує хаос, схильний до ризикованих стратегій"
        },
        "Емерджентний аналітик": {
            "high": ["AD", "KB", "RM", "EI"],
            "low": [],
            "description": "Адаптивний, технологічний, рефлексивний, аналітичний, швидко змінює стратегії, використовує AI/ML"
        },
        "Антифрагільний агент": {
            "high": ["AD", "RM", "EI", "AS"],
            "low": [],
            "description": "Виграє від стресу, любить асиметрію, використовує невизначеність як ресурс, гнучкий"
        },
        "Трендсеттер": {
            "high": ["AS", "EI", "AD"],
            "medium": ["KB"],
            "low": [],
            "description": "Впливає на думки інших, формує наративи, соціально активний, високий емоційний інтелект"
        },
        "Медіатор Духів": {
            "high": ["EI", "AS", "AD"],
            "low": ["KB"],
            "description": "Висока емпатія, інтуїція, емоційна мудрість, схильність до рефлексії, балансує емоції та раціо"
        }
    }
    

In [2]:
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
import requests
import uuid
import time
import random

class RiskToleranceTest:
    def __init__(self, questions):
        self.questions = questions
        self.current_question = 0
        self.user_answers = []
        self.scores = {"EI": 0, "KB": 0, "RM": 0, "AD": 0, "AS": 0}
        self.output = widgets.Output()
        self.radio = None
        
        # Нові атрибути
        self.user_id = str(uuid.uuid4())[:8]  # Унікальний ID
        self.test_start_time = None  # Час старту тесту
        self.response_times = []  # Час відповідей на питання
        self.demographics = {}  # Демографічні дані
        self.question_start_time = None  # Час початку поточного питання

    def start(self):
        clear_output(wait=True)
        self._show_consent_and_demographics()

    def _show_consent_and_demographics(self):
        with self.output:
            # Блок згоди
            display(HTML(
                f"<div style='border:1px solid #4f8cff; padding:20px; border-radius:8px; margin:20px 0;'>"
                f"<b>🔒 Ваш ID:</b> {self.user_id}<br>"
                f"Дані використовуються анонімно для наукового дослідження. Ви можете припинити тест будь-коли."
                f"</div>"
            ))
            
            # Демографічні питання
            age = widgets.Dropdown(options=['до 25', '26-35', '36+'], description='Вік:')
            exp = widgets.Dropdown(options=['<1 року', '1-3 роки', '>3 років'], description='Стаж:')
            market = widgets.Dropdown(options=['Крипто', 'Фондовий', 'Форекс', 'Комбо'], description='Ринок:')
            
            start_btn = widgets.Button(
                description="Почати тест", 
                style=widgets.ButtonStyle(button_color='#4f8cff'),
                layout=widgets.Layout(margin='20px auto')
            )
            
            def on_start_click(b):
                self.demographics = {
                    'age': age.value,
                    'exp': exp.value,
                    'market': market.value
                }
                self.test_start_time = time.time()
                self._show_question()
            
            start_btn.on_click(on_start_click)
            display(widgets.VBox([age, exp, market, start_btn]))

    def _show_question(self):
        with self.output:
            clear_output(wait=True)
            if self.current_question >= len(self.questions):
                self._show_results()
                return

            q = self.questions[self.current_question]
            progress = (self.current_question / len(self.questions)) * 100

            # Перемішуємо відповіді
            answers = q['answers'].copy()
            random.shuffle(answers)
            
            self.current_answers = answers  # Зберігаємо для обробки

            header = widgets.HTML(
                f"""<div class="question-container">
                    <div class="progress-container">
                        <div class="progress-fill" style="width:{progress}%"></div>
                    </div>
                    <h2>Питання {self.current_question + 1} з {len(self.questions)}</h2>
                    <div class="question-text">{q['question']}</div>
                </div>"""
            )

            self.radio = widgets.RadioButtons(
                options=[ans["text"] for ans in answers],
                layout=widgets.Layout(width='auto', max_width='900px'),
                style={'description_width': 'initial', 'font_size': '15px'}
            )
            
            next_btn = widgets.Button(
                description='Далі' if self.current_question < len(self.questions)-1 else 'Завершити',
                layout=widgets.Layout(
                    width='auto',
                    min_width='220px',
                    max_width='100%',
                    height='44px',
                    margin='24px auto 0 auto',
                    align_self='center'
                ),
                style=widgets.ButtonStyle(button_color='#4f8cff')
            )
            next_btn.on_click(self._handle_answer)
            
            display(widgets.VBox(
                [header, self.radio, next_btn],
                layout=widgets.Layout(
                    align_items='center',
                    width='100%',
                    max_width='900px',
                    margin='0 auto',
                    padding='20px 0'
                )
            ))
            
            self.question_start_time = time.time()  # Фіксуємо час початку питання

    def _handle_answer(self, b):
        with self.output:
            clear_output(wait=True)
            if self.radio.index is None:
                display(HTML("<div style='color:red; margin:20px'>❗ Оберіть відповідь!</div>"))
                self._show_question()
                return

            # Розрахунок часу відповіді
            question_time = time.time() - self.question_start_time
            self.response_times.append(round(question_time, 1))
            
            answer = self.current_answers[self.radio.index]
            for cat, val in answer['weights'].items():
                self.scores[cat] += val

            self.current_question += 1
            self._show_question()

    def _show_results(self):
        with self.output:
            clear_output(wait=True)
            archetype, description = determine_archetype(self.scores, self.questions)
            
            scores_html = '<div class="score-grid">'
            for cat, score in self.scores.items():
                scores_html += f"""
                <div class="score-item">
                    <div>{cat}</div>
                    <div class="score-value">{score}</div>
                </div>"""
            scores_html += '</div>'

            display(HTML(f"""
            <div class="question-container">
                <h2 style="color:#4f8cff; text-align:center; margin-bottom:30px;">🎯 Ваш психологічний архетип</h2>
                <div style="background:#181a1b; padding:20px; border-radius:12px;">
                    <h3 style="color:#4f8cff; margin-bottom:15px;">{archetype}</h3>
                    <p style="font-size:1.1em; line-height:1.6; color:#e0e6ef;">{description}</p>
                    {scores_html}
                </div>
            </div>
            """))
            display(self._create_feedback_form(archetype))
            self._send_to_telegram(archetype)

    def _create_feedback_form(self, archetype):
        feedback_label = widgets.HTML("<h4 style='color:#4f8cff; margin:25px 0 15px 0;'>💬 Залиште відгук про тест</h4>")
        feedback_text = widgets.Textarea(
            placeholder='Ваші враження, побажання, чи співпав результат?',
            layout=widgets.Layout(width='420px', height='100px', margin='0 auto 18px auto')
        )
        
        send_btn = widgets.Button(
            description='Надіслати в Telegram',
            layout=widgets.Layout(
                width='auto',
                min_width='250px',
                max_width='100%',
                height='50px',
                margin='24px auto 0 auto',
                align_self='center'
            ),
            style=widgets.ButtonStyle(button_color='#4f8cff')
        )
        feedback_output = widgets.Output()

        def on_send_click(b): 
            with feedback_output:
                clear_output()
                if feedback_text.value.strip():
                    self._send_feedback_to_telegram(feedback_text.value, archetype)
                    display(HTML("<span style='color:#4f8cff;'>Відгук надіслано!</span>"))
                    feedback_text.value = ''
                else:
                    display(HTML("<span style='color:#ff6b6b;'>Введіть текст відгуку!</span>"))

        send_btn.on_click(on_send_click)
        return widgets.VBox(
            [feedback_label, feedback_text, send_btn, feedback_output],
            layout=widgets.Layout(align_items='center', width='100%')
        )

    def _send_feedback_to_telegram(self, feedback_text, archetype):
        try:
            TELEGRAM_TOKEN = "8021521612:AAFHa0rQcPhgyBh7y3Q5ggsOkNCu_5RDvyo"
            CHAT_ID = "298306752"
            message = (
                f"📝 Новий відгук про тест:\n"
                f"Архетип: {archetype}\n"
                f"Бали: {self.scores}\n"
                f"Відгук: {feedback_text}"
            )
            requests.post(
                f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendMessage",
                json={"chat_id": CHAT_ID, "text": message}
            )
        except Exception as e:
            print(f"Помилка відправки: {str(e)}")

    def _send_to_telegram(self, archetype):
        try:
            TELEGRAM_TOKEN = "8021521612:AAFHa0rQcPhgyBh7y3Q5ggsOkNCu_5RDvyo"
            CHAT_ID = "298306752"
            
            test_duration = round(time.time() - self.test_start_time, 1)
            
            message = (
                f"📊 Результат тесту\n"
                f"ID: {self.user_id}\n"
                f"Вік: {self.demographics['age']}\n"
                f"Стаж: {self.demographics['exp']}\n"
                f"Ринок: {self.demographics['market']}\n"
                f"Час проходження: {test_duration} сек\n"
                f"Час на питання: {self.response_times}\n"
                f"Архетип: {archetype}\n"
                f"Бали:\n" + "\n".join([f"- {cat}: {score}" for cat, score in self.scores.items()])
            )
            
            requests.post(
                f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendMessage",
                json={"chat_id": CHAT_ID, "text": message}
            )
        except Exception as e:
            print(f"Помилка відправки: {str(e)}")

test = RiskToleranceTest(questions)
test.start()

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 125)